<a href="https://colab.research.google.com/github/aayushdebugging/GenerativeAI/blob/main/Multilingual_TTS_STT_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install PyPDF2
!pip install langchain
!pip install Pillow
!pip install pytesseract
!pip install langchain-huggingface
!pip install huggingface_hub
!pip install transformers
!pip install accelerate
!pip install bitsandbytes
!pip install langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.2/396.2 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 646.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [4]:
!sudo apt install -y tesseract-ocr
!pip install pytesseract datasets seqeval tensorboard evaluate --upgrade

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 0s (13.9 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debco

In [5]:

from google.colab import userdata
sec_key=userdata.get("HF_TOKEN")

In [6]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"]=sec_key

In [7]:

from langchain_huggingface import HuggingFaceEndpoint
from langchain import PromptTemplate, LLMChain


In [ ]:

repo_id="mistralai/Mistral-7B-Instruct-v0.3"
llm=HuggingFaceEndpoint(repo_id=repo_id,max_length=128,temperature=0.7,token = sec_key)

                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.
                    token was transferred to model_kwargs.
                    Please make sure that token is what you intended.


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [8]:
!pip install python-docx


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 4.0 MB/s eta 0:00:00


In [22]:
import requests
from langchain import PromptTemplate, LLMChain
from google.colab import files
from IPython.display import display, Image as IPImage
from PIL import Image
import pytesseract
import io
import re
from google.colab import userdata
from PyPDF2 import PdfReader
from docx import Document

# Set up the API key for SARVAM
SARVAM_API_KEY = userdata.get("sarvam_ai")

# Function to extract text from an image
def extract_text_from_image(image):
    image = Image.open(io.BytesIO(image))
    display(image)  # Display the image in the notebook
    extracted_text = pytesseract.image_to_string(image)
    return extracted_text

# Function to extract text from a PDF
def get_pdf_text(pdf_docs):
    pdf = io.BytesIO(pdf_docs)
    text = ""
    pdf_reader = PdfReader(pdf)
    for page in pdf_reader.pages:
        text += page.extract_text()
    return text

# Function to extract text from a Word document
def get_word_text(docx_docs):
    doc = Document(io.BytesIO(docx_docs))
    text = ""
    for para in doc.paragraphs:
        text += para.text + "\n"
    return text

# Function to set up the conversational chain using Hugging Face
def get_conversational_chain(question):
    prompt_template = """
    Text: {text}

    Question: {question}

    Answer: Let's think step by step and return the answer.
    """

    prompt = PromptTemplate(template=prompt_template, input_variables=["text", "question"])
    chain = LLMChain(llm=llm, prompt=prompt)
    return chain

# Function to get the answer based on the text and question
def get_answer(text, question):
    chain = get_conversational_chain(question)
    # Pass the dictionary with both 'text' and 'question'
    res = chain.run({"text": text, "question": question})
    return res

# Function to preprocess text
def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

# Function to translate text to a selected language using Sarvam.ai
def translate_text_to_language(text, target_language):
    url = "https://api.sarvam.ai/translate"
    payload = {
        "input": text,
        "source_language_code": "en-IN",
        "target_language_code": target_language,
        "speaker_gender": "Male",
        "mode": "formal",
        "model": "mayura:v1",
        "enable_preprocessing": True
    }
    headers = {
        "Content-Type": "application/json",
        "API-Subscription-Key": SARVAM_API_KEY
    }
    response = requests.post(url, json=payload, headers=headers)
    translated_text = response.json().get("translated_text", "")
    return translated_text

# Function to handle user interaction
def main():
    # Step 1: Process the document (PDF, Image, or Word)
    print("Please upload a PDF, image, or Word document.")
    uploaded = files.upload()

    if uploaded:
        for file_name in uploaded.keys():
            file_bytes = uploaded[file_name]
            file_type = file_name.split('.')[-1]

            if file_type == 'pdf':
                raw_text = get_pdf_text(file_bytes)
            elif file_type in ['jpg', 'jpeg', 'png']:
                raw_text = extract_text_from_image(file_bytes)
            elif file_type == 'docx':
                raw_text = get_word_text(file_bytes)
            else:
                print("Invalid file type.")
                return

        preprocessed_text = preprocess_text(raw_text)

        # Step 2: Language Selection
        target_language = input("Please select the target language code (e.g., 'hi-IN' for Hindi): ")
        print(f"You selected {target_language}.")

        # Step 3: Ask the user for a question
        question = input("Please enter your question: ")

        # Step 4: Get the answer from the document
        answer = get_answer(preprocessed_text, question)

        # Display the text answer
        print(f"Answer: {answer}")

        # Step 5: Translate the answer to the selected language
        translated_answer = translate_text_to_language(answer, target_language)
        print(f"Answer in {target_language}: {translated_answer}")

# Run the main function
main()


Please upload a PDF, image, or Word document.


Saving Test.docx to Test (7).docx
Please select the target language code (e.g., 'hi-IN' for Hindi): hi-IN
You selected hi-IN.
Please enter your question: कंपनी का नाम क्या है?
Answer: 1. First, we need to identify the company name.
    2. In the provided text, it is mentioned that the company is called QuickShop Online.

    Answer: QuickShop Online.
Answer in hi-IN: 1. पहला कदम कंपनी का नाम पता लगाना है।
2. दिए गए पाठ से, हम सीखते हैं कि कंपनी का नाम क्विकशॉप ऑनलाइन है।

उत्तर: कंपनी का नाम क्विकशॉप ऑनलाइन है।


In [30]:
import requests
from langchain import PromptTemplate, LLMChain
from google.colab import files
from IPython.display import display, Image as IPImage
from PIL import Image
import pytesseract
import io
import re
from google.colab import userdata
from PyPDF2 import PdfReader
from docx import Document

# Set up the API key for SARVAM
SARVAM_API_KEY = userdata.get("sarvam_ai")

# Function to extract text from an image
def extract_text_from_image(image):
    image = Image.open(io.BytesIO(image))
    display(image)  # Display the image in the notebook
    extracted_text = pytesseract.image_to_string(image)
    return extracted_text

# Function to extract text from a PDF
def get_pdf_text(pdf_docs):
    pdf = io.BytesIO(pdf_docs)
    text = ""
    pdf_reader = PdfReader(pdf)
    for page in pdf_reader.pages:
        text += page.extract_text()
    return text

# Function to extract text from a Word document
def get_word_text(docx_docs):
    doc = Document(io.BytesIO(docx_docs))
    text = ""
    for para in doc.paragraphs:
        text += para.text + "\n"
    return text

# Function to set up the conversational chain using Hugging Face
def get_conversational_chain(question):
    prompt_template = """
    Text: {text}

    Question: {question}

    Answer: Provide a concise and direct answer.
    """
    prompt = PromptTemplate(template=prompt_template, input_variables=["text", "question"])
    chain = LLMChain(llm=llm, prompt=prompt)
    return chain

# Function to get the answer based on the text and question
def get_answer(text, question):
    chain = get_conversational_chain(question)
    res = chain.run({"text": text, "question": question})
    return res

# Function to preprocess text
def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

# Function to translate text to a selected language using Sarvam.ai
def translate_text_to_language(text, target_language):
    url = "https://api.sarvam.ai/translate"
    payload = {
        "input": text,
        "source_language_code": "en-IN",
        "target_language_code": target_language,
        "speaker_gender": "Male",
        "mode": "formal",
        "model": "mayura:v1",
        "enable_preprocessing": True
    }
    headers = {
        "Content-Type": "application/json",
        "API-Subscription-Key": SARVAM_API_KEY
    }
    response = requests.post(url, json=payload, headers=headers)
    translated_text = response.json().get("translated_text", "")
    return translated_text

# Function to get the final translated answer
def get_final_answer(answer, target_language):
    translated_answer = translate_text_to_language(answer, target_language)
    return translated_answer.strip()

# Function to handle user interaction
def main():
    # Process the document (PDF, Image, or Word)
    print("Please upload a PDF, image, or Word document.")
    uploaded = files.upload()

    if uploaded:
        for file_name in uploaded.keys():
            file_bytes = uploaded[file_name]
            file_type = file_name.split('.')[-1]

            if file_type == 'pdf':
                raw_text = get_pdf_text(file_bytes)
            elif file_type in ['jpg', 'jpeg', 'png']:
                raw_text = extract_text_from_image(file_bytes)
            elif file_type == 'docx':
                raw_text = get_word_text(file_bytes)
            else:
                print("Invalid file type.")
                return

            preprocessed_text = preprocess_text(raw_text)

            # Language Selection
            language_code = input("Please select your language (e.g., 'hi-IN' for Hindi): ")
            print(f"You selected {language_code}.")

            # Question Input
            question = input("Please enter your question: ")

            # Get the answer based on the text and question
            answer = get_answer(preprocessed_text, question)

            # Get the final translated answer
            final_answer = get_final_answer(answer, language_code)
            print(f"Final Answer in {language_code}: {final_answer}")

# Run the main function
main()


Please upload a PDF, image, or Word document.


Saving Test.docx to Test (4).docx
Please select your language (e.g., 'hi-IN' for Hindi): hi-IN
You selected hi-IN.
Please enter your question:  यह कंपनी क्या करती है?
Final Answer in hi-IN: क्विकशॉप ऑनलाइन एक ऐसी कंपनी है जो ऑनलाइन खरीदारी को बढ़ाने पर ध्यान केंद्रित करती है। कंपनी के बारे में कुछ प्रश्न और उत्तर निम्नलिखित हैंः

1. कंपनी की स्थापना कब हुई थी?

उत्तर: क्विकशॉप ऑनलाइन की शुरुआत 2015 में हुई थी।

2. कंपनी का मुख्यालय कहाँ स्थित है?

उत्तर: क्विकशॉप ऑनलाइन का मुख्यालय कैलिफोर्निया के सैन फ्रांसिस्को में है।

3. कंपनी की वेबसाइट क्या है?

उत्तर: क्विकशॉप ऑनलाइन की वेबसाइट www.quickshoponline.com है।

4. कंपनी का व्यावसायिक मॉडल क्या है?

उत्तर: क्विकशॉप ऑनलाइन का व्यावसायिक मॉडल बी2सी है, जिसका अर्थ है व्यवसाय से उपभोक्ता।

5. कंपनी का मिशन वक्तव्य क्या है?

जवाबः क्विकशॉप ऑनलाइन का मिशन बयान ऐप खरीदारी के अनुभव को तेज और अधिक सहायक बनाते हुए बढ़ाना है।
